In [0]:
/* ADS_STG_SMA_CASE_PHASE_PROPERTIES_MAIN */
TRUNCATE TABLE gap_catalog.ads_etl_owner.XC_STG_SMA_CASE_PHASE_PROPERTIES_MAIN;

--select * from gap_catalog.ads_etl_owner.XC_STG_SMA_CASE_PHASE_PROPERTIES_MAIN;

INSERT INTO gap_catalog.ads_etl_owner.XC_STG_SMA_CASE_PHASE_PROPERTIES_MAIN;
     SELECT CPPROP_SOURCE_ID,
               CPPROP_SOURCE_SYSTEM_ID,
               CPPROP_SOURCE_SYS_ORIGIN,
               CPPROP_PARENT_SOURCE_ID,
               CPPTP_SOURCE_ID,
               CASETYPE,
               CPPROP_VALUE_TEXT,
               CPPROP_VALUE_NUMBER,
               CPPROP_VALUE_DATE,
               STRUCTURETYPE,
               MAPPING,
               CASE_KEY,
               CTP_KEY,
               CPPROP_KEY,
               ATTR_FP,
               CPPTP_KEY,
               CPPROP_PARENT_KEY
          FROM (WITH
                    STEP_0
                    AS
                    (
                    SELECT valid_to_date 
                    FROM (
                        SELECT ADD_MONTHS(LAST_DAY(to_date('20250911','yyyyMMdd')), pos) AS VALID_TO_DATE
                        FROM (
                            SELECT EXPLODE(
                                SEQUENCE(
                                    0, 
                                    CAST(MONTHS_BETWEEN(
                                        LAST_DAY(CURRENT_DATE()), 
                                        LAST_DAY(to_date('20250911','yyyyMMdd'))
                                    ) AS INT)
                                )
                            ) AS pos
                        )
                        WHERE MONTHS_BETWEEN(
                            LAST_DAY(CURRENT_DATE()),
                            ADD_MONTHS(LAST_DAY(to_date('20250911','yyyyMMdd')), pos)
                        ) > 0
                        UNION
                        SELECT to_date('20250911','yyyyMMdd') AS VALID_TO_DATE
                        UNION ALL
                        SELECT to_date('30000101','yyyyMMdd') AS VALID_TO_DATE)
                    ),
                    STEP_1
                    AS
                    (
                    SELECT CASE_SOURCE_ID,
                        CPPROP_SOURCE_ID,
                        CPPROP_PARENT_SOURCE_ID,
                        CPPTP_SOURCE_ID,
                        CASETYPE,
                        CTP_KEY,
                        ATTR_TYPE,
                        ATTR_BVALUE,
                        ATTR_VALUE,
                        STRUCTURETYPE,
                        MAPPING,
                        ATTR_FP,
                        CASE
                            WHEN SNAP.ATTR_TYPE = 'TEXT'
                            THEN CAST(SUBSTR(SNAP.ATTR_BVALUE, 1, 4000) AS VARCHAR(4000))
                            ELSE SUBSTR(SNAP.ATTR_VALUE, 1, 4000)
                        END AS CPPROP_VALUE_TEXT,
                        CASE
                            WHEN SNAP.ATTR_TYPE IN ('NUMBER', 'DECIMAL')
                                    AND TRIM(CAST(SUBSTR(SNAP.ATTR_VALUE, 1, 4000) AS VARCHAR(4000))) 
                                        NOT IN ('Infinity', 'NaN')
                            THEN TRY_CAST(REPLACE(TRIM(SNAP.ATTR_VALUE), ',', '.') AS DECIMAL(38, 10))
                            ELSE CAST(NULL AS DECIMAL(38, 10))
                        END AS CPPROP_VALUE_NUMBER
                    FROM gap_catalog.ads_etl_owner.STG_SMA_CASE_PHASE_PROPERTIES SNAP
                    WHERE SNAP.ATTR_VALUE IS NOT NULL
                        OR SNAP.ATTR_BVALUE IS NOT NULL
                        OR SNAP.ATTR_TYPE IN ('MAP', 'LIST')
                     ),
                    STEP_2
                    AS
                        (SELECT 
                                S1.CASE_SOURCE_ID,
                                S1.CPPROP_SOURCE_ID,
                                S1.CPPROP_PARENT_SOURCE_ID,
                                S1.CPPTP_SOURCE_ID,
                                S1.CASETYPE,
                                S1.ATTR_TYPE,
                                S1.ATTR_BVALUE,
                                S1.ATTR_VALUE,
                                S1.STRUCTURETYPE,
                                S1.MAPPING,
                                S1.ATTR_FP,
                                S1.CPPROP_VALUE_TEXT,
                                S1.CPPROP_VALUE_NUMBER,
                                NVL (C.CASE_KEY, -1)     AS CASE_KEY,
                                NVL (C.CTP_KEY, -1)      AS CTP_KEY,
                                C.CASE_START_DATE
                           FROM STEP_1  S1 
                           left join
                              (select * from gap_catalog.ads_src_20250901.CASES 
                                where CASE_SOURCE_SYSTEM_ID = 'SMA' 
                                  and CASE_SOURCE_SYS_ORIGIN = 'SMA_MONITOR_EVENTS'
                                  and CASE_VALID_TO in (select valid_to_date from  STEP_0) 
                              ) C
                         ON S1.CASE_SOURCE_ID = C.CASE_SOURCE_ID
                            )
                SELECT 
                       S2.CPPROP_SOURCE_ID            AS CPPROP_SOURCE_ID,
                       'SMA'                          AS CPPROP_SOURCE_SYSTEM_ID,
                       'SMA_MONITOR_EVENTS'           AS CPPROP_SOURCE_SYS_ORIGIN,
                       S2.CPPROP_PARENT_SOURCE_ID     AS CPPROP_PARENT_SOURCE_ID,
                       S2.CPPTP_SOURCE_ID             AS CPPTP_SOURCE_ID,
                       S2.CASETYPE                    AS CASETYPE,
                       S2.CPPROP_VALUE_TEXT           AS CPPROP_VALUE_TEXT,
                       S2.CPPROP_VALUE_NUMBER         AS CPPROP_VALUE_NUMBER,
                       CAST (NULL AS DATE)            AS CPPROP_VALUE_DATE,
                       S2.STRUCTURETYPE               AS STRUCTURETYPE,
                       S2.MAPPING                     AS MAPPING,
                       S2.CASE_KEY                    AS CASE_KEY,
                       S2.CTP_KEY                     AS CTP_KEY,
                       NVL (CPP.CPPROP_KEY, -1)       AS CPPROP_KEY,
                       S2.ATTR_FP                     AS ATTR_FP,
                       -1                             AS CPPTP_KEY,
                       -1                             AS CPPROP_PARENT_KEY
                  FROM STEP_2  S2
                       LEFT JOIN gap_catalog.ads_src_20250901.CASE_PHASE_PROPERTIES CPP
                           ON     S2.CPPROP_SOURCE_ID = CPP.CPPROP_SOURCE_ID
                              AND S2.CASE_START_DATE = CPP.CASE_START_DATE
                              AND S2.CTP_KEY = CPP.CTP_KEY
                              AND CPP.CPPROP_SOURCE_SYS_ORIGIN =
                                  'SMA_MONITOR_EVENTS'
                              AND CPP.CPPROP_SOURCE_SYSTEM_ID = 'SMA'
                              AND CPP.CPPROP_VALID_TO = DATE '3000-01-01')     
;

TRUNCATE TABLE gap_catalog.ads_owner.SMA_CASE_PHASE_PROPERTIES;

--select * from  gap_catalog.ads_owner.SMA_CASE_PHASE_PROPERTIES;

/* INSERT into SMA_CASE_PHASE_PROPERTIES*/
insert  into gap_catalog.ads_owner.SMA_CASE_PHASE_PROPERTIES trg 
 ( CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_SOURCE_ID, 
   CPPROP_PARENT_KEY, 
   CPPTP_SOURCE_ID, 
   CPPTP_KEY, 
   CASETYPE, 
   CASE_KEY, 
   CTP_KEY, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_NUMBER, 
   CPPROP_VALUE_DATE, 
   STRUCTURETYPE, 
   MAPPING, 
   ATTR_FP, 
   SMCPP_INSERTED_DATETIME, 
   SMCPP_INSERT_PROCESS_KEY, 
   SMCPP_UPDATED_DATETIME, 
   SMCPP_UPDATE_PROCESS_KEY)
select  CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_SOURCE_ID, 
   CPPROP_PARENT_KEY, 
   CPPTP_SOURCE_ID, 
   CPPTP_KEY, 
   CASETYPE, 
   CASE_KEY, 
   CTP_KEY, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_NUMBER, 
   CPPROP_VALUE_DATE, 
   STRUCTURETYPE, 
   MAPPING, 
   ATTR_FP, 
   CURRENT_TIMESTAMP() as SMCPP_INSERTED_DATETIME, 
   13165092 as SMCPP_INSERT_PROCESS_KEY, 
   CURRENT_TIMESTAMP() as SMCPP_UPDATED_DATETIME, 
   13165092 as SMCPP_UPDATE_PROCESS_KEY
  from gap_catalog.ads_etl_owner.XC_STG_SMA_CASE_PHASE_PROPERTIES_MAIN src
  ;